# 2 - Accessing the Dataset
**Estimated time to run through notebook is 20 minutes** 

This notebook is split into the following sections:
- [Load libraries, predefine some functions](#preprocessing)
- [2.1 Understanding the dataset](#understanding)
- [2.2 Subsetting the dataset](#sub)
- [2.3 Interacting with the dataset](#interact) 
- [2.4 Interacting with a table to query column descriptions](#columns)
- [Conclusion](#conclusion)

#### Resources
- Jupyter Notebooks - https://jupyter-notebook.readthedocs.io/en/stable/notebook.html The Jupyter Notebook team has lots of great documentation regarding the use of notebook. Check it out if you are interested in learning more.
- pandas for reading and slicing manifests - https://pandas.pydata.org Pandas is a python package that is regulary used throughout this workshop. You may have some extra questions or need some practice with this package. Check out the [tutorials](https://pandas.pydata.org/docs/getting_started/tutorials.html) and the [documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) for more information.
- plotly for plotting - https://plotly.com/python/getting-started/



## <a id='preprocessing'></a>Load libraries, predefine some functions

### Load libraries and predefined functions

Every chapter starts with a block of code that will import packages and functions that will be used downstream for parsing, analysis and visualization.

In [ ]:
import warnings
import pandas as pd
import plotly.express as px
import numpy as np

## <a id='understanding'></a> 2.1 Understanding the dataset

### The hiPSC Single-Cell Image Dataset
Our dataset consists of 3D live-cell images of *normal human induced pluripotent stem cells (hiPSCs)*, which are a useful human cell model system. The dataset includes 25 cell lines representing fluorescently tagged major organelles, cellular structures, and compartments. Images of these cells were obtained in 3D using spinning-disk confocal microscopes and were further processed using deep learning-based segmentation.

<img src="resources/hipsc_single_cell_image_dataset_summary.png"/>

As was described in the first chapter, the dataset consists of 25 different fluorescently tagged proteins that we use to visualize some of the majore organelles. Below you'll find a table that describes the gene, protein, a description, where in the cell the protein is generally locatated, as well the structure that the protein represents.

In [ ]:
import plotly.graph_objects as go
cell_line_anno = pd.read_csv("resources/cell_line_annotation.txt",delimiter="\t")

fig_tab = go.Figure(
    data=[go.Table(
        columnwidth=[80,80,400,80,80],
        header=dict(values=list(cell_line_anno.columns),
            fill_color='paleturquoise',
            align = 'left'),
        cells=dict(values=[cell_line_anno.Gene,cell_line_anno.Protein,cell_line_anno['Description (from Uniprot)'],cell_line_anno['Cellular Component'],cell_line_anno.Structure],
                   fill_color='lavender',
                   align='left'))

    ]
)
fig_tab.update_layout(
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=10,
        pad=4
    ),
)
fig_tab.show(renderer="jupyterlab")

### From FOVs to single-cell images
The original hiPSC Single-Cell Image dataset consists of **215,081** single-cell images which were extracted from **18,100** fields of view (FOVs) of our cell colonies. Segmentations were computed at the FOV level using the Allen Cell and Structure Segmenter (Chen et al., 2018), with code and trained models available at https://github.com/AllenCell/segmenter_model_zoo. Single cells were extracted from the FOVs using the cell segmentation results. All images were rescaled to isotropic voxel sizes by interpolating along the z dimension to upscale the voxel size from 0.108333 $\mu$m x 0.108333 $\mu$m x 0.29 $\mu$m  to 0.108333 $\mu$m x 0.108333 $\mu$m x 0.108333 $\mu$m. ROIs were determined by extending the 3D bounding box of the cell by 40 pixels in x and y, and 10 pixels in z. 

The single-cell images were further processed via a filtering in which outliers were identified based on cell and nuclear metrics e.g. cell surface area and nuclear volume. For our purposes we have removed any cells that were originally identified as an  `outlier` from our processing. The version of the dataset we're working with has outliers removed, resulting in **214,037** single-cell images. The code for this processing is available at https://github.com/AllenCell/cvapipe_analysis/tree/master/cvapipe_analysis/steps/preprocessing. You'll learn more about the cell processing steps in the next chapter.


#### Load the manifest

In order to access the dataset we have created a manifest where each row is a cell and each column has different meta-data, metrics, and paths relating to those cells. This will be the primarily way to access and manipulate the data. We begin by first loading the manifest using the pandas read_parquet function. We then set the index of the data frame using the CellId column to ensure that the CellId is propgated throughout any of our filtering steps.

In [ ]:
df = pd.read_parquet("s3://allencell-hipsc-cytodata/hackathon_manifest_17oct2022.parquet")
df.set_index(df['CellId'].astype(int), inplace=True)
print(f'Number of cells: {len(df)}')
print(f'Number of columns: {len(df.columns)}')

The manifest contains **214037** rows which represent the cells and **77** columns of metrics, metadata, and image paths which will be described later!

### Overview - FOVs per gene

The cell images came from over 18K FOVs and are scattered across the 25 different cell lines. We can parse the manifest and utilize it to understand how many FOV's were captured per gene.  The below code takes the manifest `df` and groups the data by the `gene` column. This allows us to perform any operation on our grouped data, and in this instance we'll access the`FOVId` column and count the number of FOVId's that were captured per gene.

In [ ]:
fig_fovstruc = px.bar(df.groupby(['gene']).FOVId.nunique(),text_auto=True,title="# of FOV per gene")
fig_fovstruc.update_layout(xaxis=dict(categoryorder="total descending"),overwrite=True)
fig_fovstruc.show()

The number of FOVs isn't evenly distributed between the cell lines! It ranges from almost 2000 to 150 FOVs per cell line.

The groupby function is very handy for performing operations across different grouped columns. For example we can easily replace the above code with a different column - say `cell component` to understand how many FOVs were captured for structures residing in particular cellular localizations.

In [ ]:
fig_fovstruc = px.bar(df.groupby(['Cellular Component']).FOVId.nunique(),text_auto=True,title="# of FOV per cellular localization")
fig_fovstruc.update_layout(xaxis=dict(categoryorder="total descending"),overwrite=True)
fig_fovstruc.show()

### Overview - Cells per FOV

Each FOV contains a different numbers of cells that is determined by a variety of factors including confluency (how dense the cells are on the plate) as well as colony position (some FOVs can be centered on the colony while others are at the edges of our colonies). We can quickly look at the number of cells per FOV by using our groupby function. However in this case, we'll do some more advanced filtering in order to retain an FOVId which we will use in the next step.

The code below groups the manifest again by `gene` and then again counts the number cells contained in each`FOVId` and creates a new dataframe `cell_count_per_FOV` this ultimately is used to create a dataframe that has FOVID and cell count as the columns all grouped by the `gene`. We can then plot this as a combination violin and box plot to understand the distribution of cells per FOV.

In [ ]:
cells_per_fov = (
    df.groupby('gene').FOVId 
    .value_counts()
    .to_frame(name="cell_count_per_FOV")
    .reset_index()
    .set_index('FOVId')
    .merge(
        df[['FOVId']]
        .drop_duplicates(subset=['FOVId'])
        .set_index('FOVId'),
        left_index=True,
        right_index=True
    )
    .reset_index()
)

In [ ]:
fig_cells_fov = px.violin(
    cells_per_fov,
    x=cells_per_fov.gene,
    y=cells_per_fov.cell_count_per_FOV,
    box=True,
    points='all',
    color=cells_per_fov.gene,
    hover_data=cells_per_fov,
    title="Cells per FOV for each gene"
)
fig_cells_fov.show()

Visualizing the data we can see that overall the distribution of the number of cells is consistent through the different cell lines. We can see that it FOV's generally contain between 2 and 20+ cells with a median around 14.  Though the FOV data is not included for the purposes of this task we can still visualize the FOVs as they are publically available data.  First we'll create a simple function that uses the FOVId to identify a Cell ID which we can use in our browser based cell viewer. You'll learn more ways to visualize the cells in the next chapters. But for now we can take a quick peek with the link below.

In [ ]:
def generate_url(FOV_of_interest):
    cell_id = df[df['FOVId']==FOV_of_interest].CellId.tolist()[0]
    print(f"Click this link https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id={int(cell_id)} and select the Full Field at the top of the screen")

We can hover the data from the violin plot above and select the highest point within the **ACTB** scatter plot and see that the **FOVID is 5785**. We'll use this to look at an FOV with 34 cells! 

In [ ]:
FOV_of_interest = 5785 # CHANGE ME -  Enter FOV of interest in this case 5785 which comes from the max number of cells of of ACTB
generate_url(FOV_of_interest)

The above code is just one of the many ways you'll learn how to visualize the cells that is included in **Chapter 3**.

### Overview - Cells per gene

The dataset consists of over 200K cells and based on the number distribution of FOVs we can assume that they are not evenly spraed across the 25 cell lines. We can visualize the number of cells per gene by further parsing the data frame.

In [ ]:
fig_total = px.bar(df.gene.value_counts(),text_auto=True,title="Total cells per gene",labels=dict(value="# of Cells"))
fig_total.show()

## <a id='sub'></a> 2.2 Subsetting the dataset

### Dataset Categories

The dataset is quite complex representing multiple cell lines that span different cellular structures as well cellular localizations. In addition we have **78** metadata columns that are associated with each cell! In order to help with parsing we have broken down the dataset columns into 4 Categories [`cell metadata`, `field-of-view metadata`,`cell metric`,`cell images`] with a brief description of them below:

**cell metadata**: These 10 columns contain metadata pertaing to the cells, for example the cell cycle stage (cell_stage) and the tagged structure name (Structure).

**field-of-view metadata**: These 20 columns that annotate the field-of-view including: instrument used (instrumentId) plate confluency (meta_plate_confluency) and more.

**cell metric**: These 36 columns are metrics that pertain to the cell such as angle of rotation and bounding box information, structure volume, and nuclear volume.

**cell images**: These 11 columns give the paths to various types of images including max projections and center slices.

<img src="resources/Chapter2_columns.jpeg"/>


For ease of use we've decided to use a pivot table in order to filter the columns. We'll give some examples on how this will work below. First we need to read in the pivot table.

In [ ]:
col_df = pd.read_csv("resources/hackathon_column_descriptions.csv",delimiter=",")
col_df.columns = ["column name","description","category"]
col_df.head(3)

### Creating Sub-Datasets

With the pivot table we can quickly and easily parse the data set select out specific categories of data. For example, if we wanted to see all the columns that pertained to the *cell metatdata* we would use the code below. 
We take our dataframe `df` and filter for only the `column name` that are associated with `cell metadata` in our pivot table `col_df`. 

In [ ]:
df_cell_metadata = df.filter(items=(col_df[col_df.category=="cell metadata"]['column name']))
df_cell_metadata.head(3)

The above code can be changed to any of the four categories [`cell metadata`, `field-of-view metadata`,`cell metric`,`cell images`] to quickly get the corresponding information. In section 2.4 we have verbose descriptions of our columns which will allow you to get a better understanding of what each of these values represents.

#### Compute mean volume of cells by cell line

We've already previously learned of ways to split the data based on the broad column categories such as cell_metadata. But we can also quickly operate on specific columns in the usual pandas syntax. For example the cells all have a metric for `cell_volume`. A question we may have is if all the cell lines had a simliar cell volume. To address this we simply need two columns,  1) **cell_volume**  and 2) **gene** from the manifest. Below is code to access these columns.

First we create a new dataframe with only the columns we are interested in. Then we groupby gene and calculate the mean of the volumes.

In [ ]:
# Same as 
df_cell_vol = df[['cell_volume','gene']].copy()
df_cell_vol.groupby('gene').mean()

Parsing the data this way we can see the mean cell volumes for each cell line. 

## <a id='interact'></a> 2.3 Interacting with the dataset

Throughout this hackathon you will often interact with Pandas DataFrames. For those unfamiliar with Pandas DataFrames, some helpful querying and grouping functions are explored within this sub-chapter. 

Subdatasets can be generated based upon some conditional, these conditionals follow standard boolean logic. Say for example you want to subset on very specific criteria and are interested in using only the `NUP153` cell line that were in interphase `M0` and had a `nuclear height` > 3 microns. That would be represented by the below code:

#### Filtering

In [ ]:
# Boolean Indexing
df_filtered_boolean_indexing = df[
    (df['gene'] == "NUP153") &
    (df['cell_stage'] == "M0") &
    (df['nuclear_height'] > 3)
]

df_filtered_boolean_indexing.shape

We now have a new dataframe that contains **16817** cells all of NUP153 at interphase and having a nuclear height greater than 3 microns!

Similarly you can filter catagorical variables using list indexing. This allows us to gather data across multiple different values and in this case we are creating a new data frame with `NUP153` `PXN` and `TOMM20`.

In [ ]:
# List Indexing 
value_list = [
    "NUP153",
    "PXN",
    "TOMM20",
]

df_list_index = df[df["gene"].isin(value_list)]
df_list_index.shape

#### Grouping

Being able to group data by some criteria is a useful tool for analysis. Using Pandas built in function .groupby we gather useful information on a particular slice of the data. We've already used this function quite extensively in the previous code blocks. But here we demonstrate the useful function `describe`. Say for example we quickly want to understand what the number, mean, std, min, and max of a specific metric is. Below we take a look at the `nuclear_volume` for each `gene` by simply adding .describe to the end of the column metric we are interested in.

In [ ]:
# Grouping
df_groupby = df.groupby(['gene'])
df_groupby.nuclear_volume.describe()

#### Missing Data

The dataset contains many columns that may be missing values, though sparse, this omittence can affect your process. In this subsection some simple ways to interact with empty cells are explored.

In [ ]:
# Dropping missing values by column or whole dataset
df_dropna = df.copy()
df_dropna.dropna(subset=['shape_mode_3_major_tilt'], inplace = True)
df_dropna.shape

In [ ]:
# Filling, either by single column or whole dataset 
df_fill_na = df.copy()
df_fill_na['shape_mode_3_major_tilt'] = df_fill_na['shape_mode_3_major_tilt'].fillna(0)
df_fill_na.shape

## <a id='columns'></a> 2.4 Interacting with a table to query column descriptions

The dataset contains 77 columns with important metrics which you may want to incorporate during the challenge. In this section we'll create a tiny dash app that runs within the notebook in order to display an interactive table of the columns and their descriptions. **The table is searchable** lending itself to more efficient querying.

In [ ]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html 

col_df = pd.read_csv("resources/hackathon_column_descriptions.csv",delimiter=",") #Already read in but just in case
col_df.columns = ["column name","description","category"] 

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
# Some sort of Look up method built in for looking at column definitions 
from dash import dash_table
app = JupyterDash(__name__)
server = app.server

app.layout = dash_table.DataTable(
    col_df.to_dict('records'), 
    [{"name": i, "id": i} for i in col_df.columns],
    style_data={
        'whiteSpace':'normal',
        'height': 'auto',
        'lineHeight':'15px',
        'backgroundColor': 'rgb(50,50,50)',
        'color': 'white',
    },
    style_header={
        'backgroundColor':'rgb(30,30,30)',
        'color':'white'
    },  
    style_cell={
        'textAlign':'left'
    },
    filter_action="native",
)


In [ ]:
app.run_server(mode="jupyterlab",debug=False) 

## <a id='conclusion'></a> Conclusion

In this chapter we introduced the variance dataset and demonstrated various ways to group and parse the data. In Chapter 3 you'll learn how to visualize the cells.